## 0. Libarary 불러오기 및 경로설정

In [2]:
import os
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize


from pandas import DataFrame
import torch.optim as optim
import torchvision.models as models
import math

In [3]:
# 테스트 데이터셋 폴더 경로를 지정해주세요.
train_dir = '/opt/ml/input/data/train'
test_dir = '/opt/ml/input/data/eval'

## 1. Renet152

In [4]:
model = models.resnet152()

## 1. Model 정의

In [5]:
class MyModel(nn.Module):
    def __init__(self, num_classes: int = 1000):
        super(MyModel, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(64, 32),
            nn.ReLU(inplace=True),
            nn.Linear(32, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

## 2-1. Train Dataset 정의

In [39]:
from glob import glob
class TrainDataset(Dataset):    #Dataset만 받아야 한다.
    def __init__(self, transform=''):
        self.transform = transform
        self.X = glob('/opt/ml/input/data/train/images/*/*')
        self.Y = []
        
        for i,img_dir in enumerate(self.X):
            tmp = img_dir.split("/")
            mask,gender_age = tmp[-1],tmp[-2].split("_")
            gender,age = gender_age[1],int(gender_age[-1])

            if 30<=age<60:
                age = 1
            elif age>=60:
                age = 2
            else:
                age = 0

            if gender[0]=='F':
                gender = 1
            else:
                gender = 2

            if 'incorrect' in tmp:
                mask = 1
            elif 'normal' in tmp:
                mask = 2
            else:
                mask = 0
            
            self.Y.append((mask,gender,age))


    def __getitem__(self, index):
        image = Image.open(self.X[index])

        if self.transform:
            image = self.transform(image)
        return image, torch.tensor(self.Y[index])

    def __len__(self):
        return len(self.X)

## 2-2. Test Dataset 정의

In [40]:
class TestDataset(Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
        return image

    def __len__(self):
        return len(self.img_paths)

## 3. Pre-Processing

In [41]:
transform = transforms.Compose([
    Resize((512, 384), Image.BILINEAR),
    ToTensor(),
    Normalize(mean=(0.5, 0.5, 0.5), std=(0.2, 0.2, 0.2)),
])

train_data = TrainDataset(transform=transform)
# x_data = TrainDataset()


train_loader = DataLoader(train_data, shuffle=False)

## 4. Training

In [42]:
model.conv1 = nn.Conv2d(3, 64, kernel_size=(7,7), stride=(2, 2), padding=(3, 3), bias=False)
model.fc = torch.nn.Linear(in_features=2048, out_features=3, bias=True)
torch.nn.init.xavier_uniform_(model.fc.weight)
stdv = 1. / math.sqrt(model.fc.weight.size(1))
model.fc.bias.data.uniform_(-stdv,stdv)


tensor([-0.0177,  0.0132, -0.0022])

In [49]:
model.fc

Linear(in_features=2048, out_features=3, bias=True)

In [51]:
loss_fn = torch.nn.CrossEntropyLoss()
optm = optim.Adam(model.parameters(), lr=1e-3)

running_loss = 0.0
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
for epoch in range(1000):
    for i,(input, label) in enumerate(train_loader):
        input.to(device)
        label.to(device)
        optm.zero_grad()
        output = model(input)
        print(output,label)
        loss = loss_fn(output, label)
        loss.backward()
        optm.step()

        running_loss += loss.item()

RuntimeError: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same

## 3. Inference

In [1]:
# meta 데이터와 이미지 경로를 불러옵니다.
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
testImage_dir = os.path.join(test_dir, 'images')

# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
testImage_paths = [os.path.join(testImage_dir, img_id) for img_id in submission.ImageID]




transform = transforms.Compose([
    Resize((512, 384), Image.BILINEAR),
    ToTensor(),
    Normalize(mean=(0.5, 0.5, 0.5), std=(0.2, 0.2, 0.2)),
])
dataset = TestDataset(testImage_paths, transform)

loader = DataLoader(
    dataset,
    shuffle=False
)

# 모델을 정의합니다. (학습한 모델이 있다면 torch.load로 모델을 불러주세요!)
device = torch.device('cuda')
model = MyModel(num_classes=18).to(device)
model.eval()

# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
all_predictions = []
for images in loader:
    with torch.no_grad():
        images = images.to(device)
        pred = model(images)
        pred = pred.argmax(dim=-1)
        all_predictions.extend(pred.cpu().numpy())
submission['ans'] = all_predictions

# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join(test_dir, 'submission.csv'), index=False)
print('test inference is done!')

NameError: name 'pd' is not defined